# Riconoscimento delle Entità Nominate (NER)

Questo notebook proviene dal [Curriculum AI per Principianti](http://aka.ms/ai-beginners).

In questo esempio, impareremo come addestrare un modello NER sul Dataset [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) di Kaggle. Prima di procedere, scarica il file [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) nella directory corrente.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Preparare il Dataset

Inizieremo leggendo il dataset in un dataframe. Se vuoi saperne di più sull'utilizzo di Pandas, visita una [lezione sull'elaborazione dei dati](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) nel nostro [Data Science for Beginners](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O

In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Ora dobbiamo fare lo stesso con il vocabolario. Per semplicità, creeremo un vocabolario senza tenere conto della frequenza delle parole; nella vita reale potresti voler utilizzare il vettorizzatore di Keras e limitare il numero di parole.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Dobbiamo creare un dataset di frasi per l'addestramento. Passiamo in rassegna il dataset originale e separiamo tutte le singole frasi in `X` (liste di parole) e `Y` (lista di token):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Per semplicità, aggiungeremo padding a tutte le frasi con 0 token fino alla lunghezza massima. Nella vita reale, potremmo voler utilizzare una strategia più intelligente e aggiungere padding alle sequenze solo all'interno di un minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Definizione della Rete di Classificazione dei Token

Utilizzeremo una rete bidirezionale LSTM a due livelli per la classificazione dei token. Per applicare un classificatore denso a ciascuno degli output dell'ultimo livello LSTM, utilizzeremo la costruzione `TimeDistributed`, che replica lo stesso livello denso su ciascuno degli output dell'LSTM a ogni passo:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Nota che qui stiamo specificando esplicitamente `maxlen` per il nostro dataset - nel caso in cui volessimo che la rete sia in grado di gestire sequenze di lunghezza variabile, dovremmo essere un po' più ingegnosi nella definizione della rete.

Ora alleniamo il modello. Per velocità, ci limiteremo ad allenare per una sola epoca, ma potresti provare ad allenare per un periodo di tempo più lungo. Inoltre, potresti voler separare una parte del dataset come dataset di addestramento, per osservare l'accuratezza della validazione.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Testare il Risultato

Vediamo ora come funziona il nostro modello di riconoscimento delle entità su una frase di esempio:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Conclusione

Anche un semplice modello LSTM mostra risultati ragionevoli per il NER. Tuttavia, per ottenere risultati molto migliori, potresti voler utilizzare grandi modelli linguistici pre-addestrati come BERT. L'addestramento di BERT per il NER utilizzando la libreria Huggingface Transformers è descritto [qui](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
